<a href="https://colab.research.google.com/github/JHyunjun/SNU/blob/main/Tabular_Q%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you run in jupyter, turn 

```
colab = False
```



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
colab = True
if colab:
    !pip install gym pyvirtualdisplay > /dev/null 2>&1
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !apt-get update > /dev/null 2>&1
    !apt-get install cmake > /dev/null 2>&1
    !pip install --upgrade setuptools 2>&1
    !pip install ez_setup > /dev/null 2>&1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [4]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    
    %cd /content/drive/MyDrive/Colab Notebooks/snu/8주_RL/강의자료/실습강의/day2/q_learning
    !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/snu/8주_RL/강의자료/실습강의/day2/q_learning
assets			     main.py	       schedule.py
compare.py		     optimum.npy       table.npy
day2_tabular_q_learning.pdf  optimum.png       table.py
discrete_pendulum.py	     plot.py	       test.py
learned_value.png	     q_learning.ipynb  trajectory.png


# Tabular Q-learning Practice

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from discrete_pendulum import DiscretePendulumEnv

# 0. Basic definitions

## 0.0. Q-table

In [6]:
class QTable:
    def __init__(self, num_states, num_actions, gamma=0.99, pth=None):
        self.gamma = gamma
        if pth is None:
            self.Q = -300. * np.ones(shape=(num_states, num_actions))
        else:
            self.Q = np.load(pth, allow_pickle=True)

    def update(self, state, action, reward, next_state, alpha):
        # update Q-table according to the following update rule:
        # Q(s, a) <- Q(s, a) + alpha * (r + gamma max_{a'} Q(s', a') - Q(s, a))
        target = reward + self.gamma * np.max(self.Q[next_state]) - self.Q[state, action]
        # TODO2 : Build incremental update of Q-function
        self.Q[state, action] += alpha * target

    def act(self, state):
        return np.argmax(self.Q[state])

    def save(self, pth=None):
        if pth is None:
            pth = './table.npy'
        np.save(pth, self.Q)

    @property
    def value_ftn(self):
        return np.max(self.Q, axis=1)

## 0.1. Stepsize rule & Exploration Schedule

In [7]:
class LinearExplorationSchedule:
    def __init__(self, rollout_len, initial_epsilon=1., final_epsilon=0.02):
        # linear exploration schedule
        self.decrement = (initial_epsilon - final_epsilon) / rollout_len
        self.initial_epsilon = initial_epsilon
        self.final_epsilon = final_epsilon

    def __call__(self, t):
        # define this as callable object so that the schedule is stateless
        return max(self.initial_epsilon - t * self.decrement, self.final_epsilon)
    
# TODO : try exponentially decaying schedule!
class ExponentialExplorationSchedule:
    def __init__(self, decay_rate, initial_epsilon=1., final_epsilon=0.02):
        self.decay_rate = decay_rate
        self.initial_epsilon = initial_epsilon
        self.final_epsilon = final_epsilon

    def __call__(self, t):
        epsilon = self.decay_rate ** t * self.initial_epsilon
        return max(epsilon, self.final)

# 1. Q-learning

In [8]:
env = DiscretePendulumEnv()
num_states = env.observation_space.n
num_actions = env.action_space.n
gamma = 0.99

learner = QTable(num_states=env.observation_space.n, num_actions=env.action_space.n, gamma=gamma)
rollout_len = 1000000


epsilon_schedule = LinearExplorationSchedule(rollout_len, final_epsilon=0.4)
# TODO : alternative
# \epsilon = ExponentialExplorationSchedule(decay_rate=0.99, final_epsilon=0.02)

checkpoint_interval = rollout_len // 20

In [9]:
class VisitCountStepsizeSchedule:
    def __init__(self, deg=1.0):
        # polynomial stepsize schedule : $\Theta(N_t(s, a)^{-d})$
        # $N_t(s, a)$ is the number of visits of (s, a)-pair until step t
        # to satisfy Robbins-Monro condition, d must satisfy $d \in (1/2, 1]$
        assert .5 < deg <= 1
        self.deg = deg

    def __call__(self, n):
        return 1. / ((n + 1.) ** self.deg)
    
visit_count = np.zeros(shape=(num_states, num_actions))     # save visit counts N(s, a) of all state-action pairs
alpha_schedule = VisitCountStepsizeSchedule(deg=0.5001)

s = env.reset()
for t in tqdm(range(rollout_len + 1)):
    u = np.random.rand()
    if u < epsilon_schedule(t):
        a = env.action_space.sample()
    else:
        a = learner.act(state=s)

    s_next, r, _, _ = env.step(action=a)
    
    # TODO3 : Fill the below line which determines the stepsize. 
    n = visit_count[s, a]
    learner.update(state=s, action=a, reward=r, next_state=s_next, alpha=alpha_schedule(n))
    visit_count[s, a] += 1
    s = s_next
    if t % checkpoint_interval == 0:
        learner.save()

100%|██████████| 1000001/1000001 [05:42<00:00, 2918.82it/s]


# 2. Let's see if the Q-function is learned properly!

## 2.0. Visualization

In [10]:
if colab:
  import gym
  from gym.wrappers import Monitor
  import glob
  import io
  import base64
  from IPython.display import HTML
  from pyvirtualdisplay import Display
  from IPython import display as ipythondisplay

  display = Display(visible=0, size=(1400, 900))
  display.start()

  def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
    else: 
      print("Could not find video")
      

  def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

  env = wrap_env(env)

ImportError: ignored

In [ ]:
ep_len = 400
# test learned result!
trajectory = np.zeros((ep_len, 2))      # store continuous states
reward = 0.

np.random.seed(2021)
env = DiscretePendulumEnv()
if colab:
  env = wrap_env(env)
s = env.reset(deterministic=True)

for t in range(ep_len):
    trajectory[t] = np.copy(env.x)
    a = learner.act(s)
    s, r, _, _ = env.step(a)
    reward += (gamma ** t) * r
    env.render()
print('total reward =', reward)
env.close()

if colab:
  show_video()

## 2.1. Trajectory Plot

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 10), sharex=True)
x = env.dt * np.arange(ep_len)
ylabels = [r'$\theta$ (rad)', r'$\dot\theta$ (rad/s)']
ax[1].set_xlabel(r'$t$ (s)', fontsize=20)
ax[0].set_ylim(-np.pi, np.pi)
ax[1].set_ylim(-8., 8.)
ax[0].set_yticks([-np.pi, -np.pi / 2, 0, np.pi / 2, np.pi])
ax[0].set_yticklabels([r'$-\pi$', r'$-\pi/2$', r'$0$', r'$\pi/2$', r'$\pi$'])
ax[1].set_yticks([-8, -4, 0, 4, 8])
for i in range(2):
    ax[i].plot(x, trajectory[:, i])
    ax[i].set_xlim(0, x[-1])
    ax[i].grid(True)
    ax[i].set_ylabel(ylabels[i], fontsize=20)
    ax[i].tick_params(axis='both', which='major', labelsize=18)
fig.tight_layout()

# 2.2. Q-learning Result vs Optimal Solution

In [ ]:
optimal_agent = QTable(num_states=env.observation_space.n, num_actions=env.action_space.n, pth='./optimum.npy')
if colab:
  env = wrap_env(DiscretePendulumEnv())

In [ ]:
ep_len = 400
# test learned result!
trajectory = np.zeros((ep_len, 2))      # store continuous states
reward = 0.

s = env.reset(deterministic=True)

for t in range(ep_len):
    trajectory[t] = np.copy(env.x)
    a = optimal_agent.act(s)
    s, r, _, _ = env.step(a)
    reward += (gamma ** t) * r
    env.render()
print('total reward =', reward)
env.close()

if colab:
  show_video()